<center><h1>TPS April 2022</h1></center>
<center><h1>Easy EDA</h1></center>

# Simple Setup

In [ ]:
%reset -sf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

!pip install bloxs

# Data

In [ ]:
from pandas import read_csv, DataFrame, Series, IndexSlice
#from pathlib import Path
#list(Path('/kaggle/input').rglob('*.*'))

train = read_csv('/kaggle/input/tabular-playground-series-apr-2022/train.csv')
y_train = read_csv('/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv')
test = read_csv('/kaggle/input/tabular-playground-series-apr-2022/test.csv')

# Reduce memory

In [ ]:
from numpy import float64, float32, int64, int32, dtype

def reduce_mem(df):
    df = df.copy()
    
    map_dtypes = {'int': dtype(int64), 'float': dtype(float32)}
    
    for col in df:
        if df[col].dtype == dtype(int64):
            df[col] = df[col].astype(int32)
        if df[col].dtype == dtype(float64):
            df[col] = df[col].astype(float32)
    return df

train = reduce_mem(train)
test = reduce_mem(test)

# EDA

## <em>How many Subjects, Sequences and Steps in Train?</em>

In [ ]:
from bloxs import B

B(train['subject'].nunique(), '😏(Subjects)')
B(train['sequence'].nunique(), '⏫(Sequences)')
B(train['step'].nunique(), '🔀(Steps)')

## <em>But, does each Subject have the same amount of Sequences?</em>

In [ ]:
subj_seq = train[['sequence', 'subject']].pivot_table(index='subject', aggfunc='count') / 60
subj_seq

## <em>Can I see that with more detail please? With Test?</em>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('talk')

subj_seq = train[['subject', 'sequence']].pivot_table(index='subject', aggfunc='count') / 60

fig, ax = plt.subplots(1, 2, figsize=(12,8), sharex=True, sharey=True, constrained_layout=True)

subj_seq_ = test[['subject', 'sequence']].pivot_table(index='subject', aggfunc='count') / 60

_ = ax[0].hist(subj_seq, bins=int(len(subj_seq)**0.5))
_ = ax[1].hist(subj_seq_, bins=int(len(subj_seq)**0.5))

## <em>Do we have Subjects in Train also present in Test i.e. duplicates?<em>

In [ ]:
from pandas import Series

train_subj = Series(train['subject'].unique())
test_subj = Series(test['subject'].unique())

dupl = int(train_subj.isin(test_subj).sum())

B(dupl, '😏(Duplicate Subjects)')

## <em>Are States (Y) balanced?</em>

In [ ]:
# Small helper

from pandas import merge

train = merge(train, y_train, left_on='sequence', right_on='sequence')

In [ ]:
# Are States (Y) balanced?

state_0 = f"{int(round((train['state']==0).sum() / len(train)*100))}"
B('~'+state_0+'%', 'State 0', progress=state_0)
state_1 = f"{int(round((train['state']==1).sum() / len(train)*100))}"
B('~'+state_1+'%', 'State 1', progress=state_1)

f'Seems pretty balanced globally'

## <em>Is there a way to visualize State (Y-variable) mean profiles?</em>

In [ ]:
# Is there a way to visualize State (Y-variable) profile?

sensors = train.columns[train.columns.str.contains('sensor')]

steps_0 = train[train['state']==0].pivot_table(
    index=['step'],
    values=sensors,
    aggfunc='mean'
)

steps_1 = train[train['state']==1].pivot_table(
    index=['step'],
    values=sensors,
    aggfunc='mean'
)

fig, ax = plt.subplots(figsize=(10,8))
_ = ax.plot(steps_0.mean(axis=1), label='State 0')
_ = ax.plot(steps_1.mean(axis=1), label='State 1')
_ = ax.plot([0]*len(steps_0), c='black')
_ = ax.set_title("States 0|1 (mean) Profile per step")
_ = ax.set_xlabel('Step')
_ = ax.set_ylabel('Value')
_ = ax.legend()

f'Even by averaging data, there is a clear distinction between states 0 and 1 profiles'

## <em>Can we add more detail to that? How about bands?</em>

In [ ]:
# Can we add more detail to that?

fig, ax = plt.subplots(1, 2, figsize=(15,8), sharey=True, constrained_layout=True)
_ = ax[0].plot(steps_0.median(axis=1), label='PCT 50%', c='blue')
_ = ax[0].plot(steps_0.quantile(0.9, axis=1), label='PCT 90%', c='blue', alpha=0.25)
_ = ax[0].plot(steps_0.quantile(0.8, axis=1), label='PCT 80%', c='blue', alpha=0.5)
_ = ax[0].plot(steps_0.quantile(0.2, axis=1), label='PCT 20%', c='blue', alpha=0.5)
_ = ax[0].plot(steps_0.quantile(0.1, axis=1), label='PCT 10%', c='blue', alpha=0.25)
_ = ax[0].plot([0]*len(steps_0), c='black')
_ = ax[0].legend()
_ = ax[0].set_xlabel('Step')
_ = ax[0].set_ylabel('Value')
_ = ax[0].set_title('State 0')

_ = ax[1].plot(steps_1.median(axis=1), label='PCT 50%', c='orange')
_ = ax[1].plot(steps_1.quantile(0.9, axis=1), label='PCT 90%', c='orange', alpha=0.25)
_ = ax[1].plot(steps_1.quantile(0.8, axis=1), label='PCT 80%', c='orange', alpha=0.5)
_ = ax[1].plot(steps_1.quantile(0.2, axis=1), label='PCT 20%', c='orange', alpha=0.5)
_ = ax[1].plot(steps_1.quantile(0.1, axis=1), label='PCT 10%', c='orange', alpha=0.25)
_ = ax[1].plot([0]*len(steps_0), c='black')
_ = ax[1].set_xlabel('Step')
_ = ax[1].legend()
_ = ax[1].set_title('State 1')

_ = fig.suptitle('States 0|1 (mean) Profile per step with Quantiles')

f'It seems that State 0 has more extreme negative values of sensors'

## <em>Can we see that but through Subjects instead of Steps?</em>

In [ ]:
steps_0 = train[train['state']==0].pivot_table(
    index=['subject'],
    values=sensors,
    aggfunc='mean'
)

steps_1 = train[train['state']==1].pivot_table(
    index=['subject'],
    values=sensors,
    aggfunc='mean'
)

fig, ax = plt.subplots(2, 1, figsize=(15,10), sharey=True, constrained_layout=True)
_ = ax[0].plot(steps_0.median(axis=1), label='PCT 50%', c='blue')
_ = ax[0].plot(steps_0.quantile(0.9, axis=1), label='PCT 90%', c='blue', alpha=0.25)
_ = ax[0].plot(steps_0.quantile(0.8, axis=1), label='PCT 80%', c='blue', alpha=0.5)
_ = ax[0].plot(steps_0.quantile(0.2, axis=1), label='PCT 20%', c='blue', alpha=0.5)
_ = ax[0].plot(steps_0.quantile(0.1, axis=1), label='PCT 10%', c='blue', alpha=0.25)
_ = ax[0].plot([0]*len(steps_0), c='black')
_ = ax[0].legend()
_ = ax[0].set_ylabel('Value')
_ = ax[0].set_title('State 0')

_ = ax[1].plot(steps_1.median(axis=1), label='PCT 50%', c='orange')
_ = ax[1].plot(steps_1.quantile(0.9, axis=1), label='PCT 90%', c='orange', alpha=0.25)
_ = ax[1].plot(steps_1.quantile(0.8, axis=1), label='PCT 80%', c='orange', alpha=0.5)
_ = ax[1].plot(steps_1.quantile(0.2, axis=1), label='PCT 20%', c='orange', alpha=0.5)
_ = ax[1].plot(steps_1.quantile(0.1, axis=1), label='PCT 10%', c='orange', alpha=0.25)
_ = ax[1].plot([0]*len(steps_0), c='black')
_ = ax[1].set_ylabel('Value')
_ = ax[1].set_xlabel('Subject')
_ = ax[1].legend()
_ = ax[1].set_title('State 1')

_ = fig.suptitle('States 0|1 (mean) Profile per Subject with Quantiles')

f'It seems that subjects at State 1 have more volatile sensors measurements...'

## <em>Ok, enough about Y, what about the Features? Say Correlations</em>

In [ ]:
from numpy import triu

corrs = train[sensors].corr()
mask = triu(corrs)
cmap = sns.diverging_palette(230, 20, as_cmap=True)

fig, ax = plt.subplots(figsize=(15,10))
_ = sns.heatmap(corrs, mask=mask, cmap=cmap, vmax=.3, center=0, ax=ax,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

f'There doesn\'t seem to be high multicolinearity between features'
f'So, OLS-type of regressions won\'t suffer much, but still always good to regularize'
f'In presence of multicolinearity, OLS-type of regressions struggle to correctly estimate their coefficients'
f'i.e. estimated coefficients will be unstable, try comparing coefficients from 2 similar (multicolinear) datasets'
f'If present, estimator will overfit leading to (highly variant) incorrect predictions'
f'On the contrary, if we regularize a bit, we fight the multicolinearity,'
f'Reducing variance (as well as a bit of bias) but leading to way better predictions,'

## <em>How can we assess the Features by Step? More profiles.</em>

## 4 Profiles

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from pandas import Series

pv = train.drop(['subject', 'state'], axis=1).pivot_table(index=['step'], columns=['sequence']).T

profiles_num = 4

mbk = MiniBatchKMeans(n_clusters=profiles_num)
_ = mbk.fit(pv)
clus = Series(mbk.predict(pv))

fig, ax = plt.subplots(int(profiles_num**0.5), int(profiles_num**0.5), figsize=(12,12), constrained_layout=True, sharex=True, sharey=True)
axs = ax.ravel()
for ax, clus_ in zip(axs, clus.unique()):
    mask = clus == clus_
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0))
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0) + pv.reset_index(drop=True).loc[mask, :].std(axis=0), c='navy')
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0) - pv.reset_index(drop=True).loc[mask, :].std(axis=0), c='navy')
    _ = ax.set_title(f'Sensor Profile {clus_}')

# 9 Profiles

In [ ]:
# Only steps at index

from sklearn.cluster import MiniBatchKMeans
from pandas import Series

pv = train.drop(['subject', 'state'], axis=1).pivot_table(index=['step'], columns=['sequence']).T

profiles_num = 9

mbk = MiniBatchKMeans(n_clusters=profiles_num)
_ = mbk.fit(pv)
clus = Series(mbk.predict(pv))

fig, ax = plt.subplots(int(profiles_num**0.5), int(profiles_num**0.5), figsize=(12,12), constrained_layout=True, sharex=True, sharey=True)
axs = ax.ravel()
for ax, clus_ in zip(axs, clus.unique()):
    mask = clus == clus_
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0))
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0) + pv.reset_index(drop=True).loc[mask, :].std(axis=0), c='navy')
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0) - pv.reset_index(drop=True).loc[mask, :].std(axis=0), c='navy')
    _ = ax.set_title(f'Sensor Profile {clus_}')

## 16 Profiles

In [ ]:
# Only steps at index

from sklearn.cluster import MiniBatchKMeans
from pandas import Series

pv = train.drop(['subject', 'state'], axis=1).pivot_table(index=['step'], columns=['sequence']).T

profiles_num = 16

mbk = MiniBatchKMeans(n_clusters=profiles_num)
_ = mbk.fit(pv)
clus = Series(mbk.predict(pv))

fig, ax = plt.subplots(int(profiles_num**0.5), int(profiles_num**0.5), figsize=(12,12), constrained_layout=True, sharex=True, sharey=True)
axs = ax.ravel()
for ax, clus_ in zip(axs, clus.unique()):
    mask = clus == clus_
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0))
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0) + pv.reset_index(drop=True).loc[mask, :].std(axis=0), c='navy')
    _ = ax.plot(pv.reset_index(drop=True).loc[mask, :].mean(axis=0) - pv.reset_index(drop=True).loc[mask, :].std(axis=0), c='navy')
    _ = ax.set_title(f'Sensor Profile {clus_}')

## <em>And, how can we assess the Features by Sequence? Does data change in time?</em>

In [ ]:
# Sensor 00 DGP's

from numpy import arange
idx = IndexSlice

def pivot_dgps(df, is_test=False):
    from sklearn.preprocessing import StandardScaler, RobustScaler

    # pivot dgps by columns
    if not is_test:
        df = df.drop('state', axis=1)
    df = df.pivot_table(
        index=['sequence'],
        columns=['step'],
    ).T.unstack(0)

    # Prepr
    #std = StandardScaler()
    #df = DataFrame(std.fit_transform(df.values), index=df.index, columns=df.columns)

    
    # Back to original format
    #df = df.T.stack().to_frame().unstack(2).reset_index()
    #if not is_test:
    #    df.columns = train.drop('state', axis=1).columns
    #else:
    #    df.columns = test.columns

    from gc import collect
    collect()
    
    return df

pv = pivot_dgps(train)

# Plot
fig, ax = plt.subplots(int(len(sensors)**0.5)+1, 
                       int(len(sensors)**0.5)+1,
                       figsize=(12,8),
                       sharex=True, 
                       sharey=True,
                       constrained_layout=True)
axs = ax.ravel()

for sensor, ax in zip(sensors, axs):
    temp = pv.loc[:, idx[:, sensor]]
    for i, col in enumerate(temp):
        _ = ax.plot(arange(len(temp[col]))+i, temp[col], linewidth=1, color='blue', alpha=0.05)
        if i >= 200: break

# Work in progress...


### Feel free to upvote if you like. Thanks